# Graph quick start

In [3]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


In [5]:
import os


apikey=os.environ["OPENAI_API"] 
os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"

os.environ["OPENAI_API_BASE"] ="https://api.zhizengzeng.com/v1/"
os.environ["TAVILY_API_KEY"] = "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"

llm = ChatOpenAI(model="gpt-3.5-turbo",api_key=apikey)


In [6]:
memory = SqliteSaver.from_conn_string(":memory:")

In [7]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


In [8]:
graph_builder = StateGraph(State)

In [9]:
tool = TavilySearchResults(max_results=2)
tools = [tool]

llm_with_tools = llm.bind_tools(tools)


In [10]:


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}



In [11]:

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

If we don't interrupt the tools , the snapshot.next will be action !  it will use the web to search the tool_arg_query! (langgraph)

In [12]:
graph = graph_builder.compile(
    checkpointer=memory,
    # This is new!
    interrupt_before=["tools"],
    # Note: can also interrupt __after__ actions, if desired.
    # interrupt_after=["tools"]
)

In [13]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}
# The config is the **second positional argument** to stream() or invoke()!


In [14]:
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_sI0ZPFymha3lpk0h9kseZ0hv)
 Call ID: call_sI0ZPFymha3lpk0h9kseZ0hv
  Args:
    query: LangGraph


In [15]:
snapshot = graph.get_state(config)

In [32]:
existing_message = snapshot.values["messages"][-1]
existing_message.pretty_print()



================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_sI0ZPFymha3lpk0h9kseZ0hv)
 Call ID: call_sI0ZPFymha3lpk0h9kseZ0hv
  Args:
    query: LangGraph


In [30]:
print(f"tool_id :{existing_message.tool_calls[0]['id']}")
print(f'messages_id :{existing_message.id}')

tool_id :call_sI0ZPFymha3lpk0h9kseZ0hv
messages_id :run-28b0d690-cc3b-487e-83c2-2f4a330fc915-0


If we need't to intercede, we can pass None into the process

In [ ]:
# # `None` will append nothing new to the current state, letting it resume as if it had never been interrupted
# events = graph.stream(None, config, stream_mode="values")
# for event in events:
#     if "messages" in event:
#         event["messages"][-1].pretty_print()

If we need to intercede ,we can  provide the correct the response in the below

In [26]:
from langchain_core.messages import AIMessage
from langchain_core.messages import ToolMessage
answer = (
    "LangGraph is a library for building stateful, multi-actor applications with LLMs."
)
new_messages = [
    # The LLM API expects some ToolMessage to match its tool call. We'll satisfy that here.
    ToolMessage(content=answer, tool_call_id=existing_message.tool_calls[0]["id"]),
    # And then directly "put words in the LLM's mouth" by populating its response.
    AIMessage(content=answer),
]



In [27]:
new_messages[-1].pretty_print()

================================== Ai Message ==================================

LangGraph is a library for building stateful, multi-actor applications with LLMs.


In [28]:

graph.update_state(
    # Which state to update
    config,
    # The updated values to provide. The messages in our `State` are "append-only", meaning this will be appended
    # to the existing state. We will review how to update existing messages in the next section!
    {"messages": new_messages},
)


{'configurable': {'thread_id': '1',
  'thread_ts': '1ef3df46-0b06-650e-8003-d2a519326452'}}

In [29]:
print("\n\nLast 2 messages;")
print(graph.get_state(config).values["messages"][-2:])



Last 2 messages;
[ToolMessage(content='LangGraph is a library for building stateful, multi-actor applications with LLMs.', id='d7bcbda0-fd60-447f-b311-3b8a974e9678', tool_call_id='call_sI0ZPFymha3lpk0h9kseZ0hv'), AIMessage(content='LangGraph is a library for building stateful, multi-actor applications with LLMs.', id='7d725dbe-bfdd-4953-ad6e-b79d2aaeb1e7')]


replace the existing messages use the same message id ! 

In [33]:
# first we create another thread 
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "2"}}  # we'll use thread_id = 2 here
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_uf4L9qwskuqEfUwaGMuKQGiJ)
 Call ID: call_uf4L9qwskuqEfUwaGMuKQGiJ
  Args:
    query: LangGraph


In [34]:
from langchain_core.messages import AIMessage

snapshot = graph.get_state(config)
existing_message = snapshot.values["messages"][-1]


In [35]:
print("Original")
print("Message ID", existing_message.id)
print(existing_message.tool_calls[0])

Original
Message ID run-679dab8a-cd68-40c7-9613-666863d938b6-0
{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph'}, 'id': 'call_uf4L9qwskuqEfUwaGMuKQGiJ'}


In [40]:
new_tool_call = existing_message.tool_calls[0].copy()
new_tool_call["args"]["query"] = "LangGraph human-in-the-loop workflow"
new_message = AIMessage(
    content=existing_message.content,
    tool_calls=[new_tool_call],
    # Important! The ID is how LangGraph knows to REPLACE the message in the state rather than APPEND this messages
    id=existing_message.id,
)

print(new_message.pretty_print())

================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_uf4L9qwskuqEfUwaGMuKQGiJ)
 Call ID: call_uf4L9qwskuqEfUwaGMuKQGiJ
  Args:
    query: LangGraph human-in-the-loop workflow
None


In [41]:
print("Updated")
print(new_message.tool_calls[0])
print("Message ID", new_message.id)


Updated
{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph human-in-the-loop workflow'}, 'id': 'call_uf4L9qwskuqEfUwaGMuKQGiJ'}
Message ID run-679dab8a-cd68-40c7-9613-666863d938b6-0


In [42]:
graph.update_state(config, {"messages": [new_message]})

print("\n\nTool calls")
graph.get_state(config).values["messages"][-1].tool_calls



Tool calls


[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph human-in-the-loop workflow'},
  'id': 'call_uf4L9qwskuqEfUwaGMuKQGiJ'}]

resume the graph 

In [43]:
events = graph.stream(None, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://github.com/langchain-ai/langgraph/blob/main/examples/human-in-the-loop.ipynb", "content": "Build resilient language agents as graphs. Contribute to langchain-ai/langgraph development by creating an account on GitHub."}, {"url": "https://blog.langchain.dev/human-in-the-loop-with-opengpts-and-langgraph/", "content": "To do this we can model the interaction between the actors as happening across multiple discrete steps, when one actor hands off work to another actor, that results in the scheduling of the next step of the computation, and so on, until no more actors hand off work to others, and we\u2019ve reached the final result.\n And just as a human team needs more coordination than one person working by themselves, an application with multiple actors needs a coordination layer to\nMulti-step\nAs each actor hands off work to another (eg. an LLM prompt aski

Check that the model has answered the follow-up questions according to the modified message

In [44]:
events = graph.stream(
    {
        "messages": (
            "user",
            "Remember what I'm learning about?",
        )
    },
    config,
    stream_mode="values",
)

In [45]:
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Remember what I'm learning about?
================================== Ai Message ==================================

You are learning about LangGraph, specifically focusing on its human-in-the-loop workflow. If you have any specific questions or need further information on this topic, feel free to ask!
